In [ ]:
# The below code is for the dataset from: 
# Health Statistics. (2023). NCHS drug poisoning mortality by county, United States. U.S. Department of Health & Human Services. 
# Retrieved September 8, 2024, from https://catalog.data.gov/dataset/nchs-drug-poisoning-mortality-by-county-united-states
# Out of Elastic Net, XGBoost, and LightGBM modeling, the best performing model was through Elastic Net, with the addition of 3 lag features for death rate and population, for predicting the log-transformed death rate (running the model wihtout the log-transformation has similar results: a slightly lower MAE, but slightly comparitively less accurate results for the other metrics).
# As the Elastic Net model showed the most accurate performance, the code is posted for its permutation feature importance and for a plot with the average actual test values versus the model's average predicted test values per test year.
# For demonstartion purposes, the code for the best performing XGBoost (which utilizes 1 lag) and LightGBM (3 lags) models (both without any log-transformations of the target variable) are also posted below. 

#Elastic Net

# Required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import ElasticNet
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance


In [ ]:
# Load and preprocess the dataset
file_path = 'NCHS_-_Drug_Poisoning_Mortality_by_County__United_States.csv'
df = pd.read_csv(file_path)

df = df.drop(columns=['FIPS State', 'FIPS', 'Standard Deviation', 'Lower Confidence Limit', 'Upper Confidence Limit'])
df = df.sort_values(by=['Year', 'County'])

# Apply log transformation to 'Model-based Death Rate' to prepare for lagged features
df['Log_Death_Rate'] = np.log(df['Model-based Death Rate'])

# Create lag variables for 'Log_Death_Rate' and 'Population'
df['Death_Rate_lag_1'] = df.groupby('County')['Log_Death_Rate'].shift(1)
df['Population_lag_1'] = df.groupby('County')['Population'].shift(1)

df['Death_Rate_lag_2'] = df.groupby('County')['Log_Death_Rate'].shift(2)
df['Population_lag_2'] = df.groupby('County')['Population'].shift(2)

df['Death_Rate_lag_3'] = df.groupby('County')['Log_Death_Rate'].shift(3)
df['Population_lag_3'] = df.groupby('County')['Population'].shift(3)

# Drop rows with NaN in lagged columns
df = df.dropna(subset=['Death_Rate_lag_1', 'Population_lag_1', 
                       'Death_Rate_lag_2', 'Population_lag_2', 
                       'Death_Rate_lag_3', 'Population_lag_3'])

# One-hot encode categorical columns
onehot_encoder = OneHotEncoder(sparse_output=False, drop=None)
categorical_cols = ['State', 'Urban/Rural Category', 'Census Division']
encoded_df = pd.DataFrame(onehot_encoder.fit_transform(df[categorical_cols]))
encoded_df.columns = onehot_encoder.get_feature_names_out(categorical_cols)

df = df.reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)
df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

# Specify exogenous features and target variable
exogenous_features = [col for col in df.columns if col not in ['Log_Death_Rate', 'County', 'Model-based Death Rate']]
target_variable = 'Log_Death_Rate'

In [ ]:
# Splitting based on year ranges
train_df = df[df['Year'].between(2006, 2017)]  
test_df = df[df['Year'].between(2018, 2021)]  

# Define features (X) and target (y) for each split
X_train = train_df[exogenous_features]
y_train = train_df[target_variable]

X_test = test_df[exogenous_features]
y_test = test_df[target_variable]


X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [ ]:
# Initialize scalers for population and death rate features
scaler_population = MinMaxScaler()
scaler_death_rate_features = MinMaxScaler()
scaler_death_rate_target = MinMaxScaler()

# Define population and death rate features (including lags)
population_features = ['Population', 'Population_lag_1', 'Population_lag_2', 'Population_lag_3']
death_rate_features = ['Death_Rate_lag_1', 'Death_Rate_lag_2', 'Death_Rate_lag_3']

# Fit and transform population features in X_train using scaler_population
X_train[population_features] = scaler_population.fit_transform(X_train[population_features])

# Fit and transform death rate lag features in X_train using scaler_death_rate_features
X_train[death_rate_features] = scaler_death_rate_features.fit_transform(X_train[death_rate_features])

# Fit and transform 'Model-based Death Rate' for y_train using scaler_death_rate_target
y_train = scaler_death_rate_target.fit_transform(y_train.to_numpy().reshape(-1, 1))

# Apply the same transformations to X_test and y_test
X_test[population_features] = scaler_population.transform(X_test[population_features])
X_test[death_rate_features] = scaler_death_rate_features.transform(X_test[death_rate_features])
y_test = scaler_death_rate_target.transform(y_test.to_numpy().reshape(-1, 1))


In [ ]:
# Define expanding window cross-validation
train_years = np.sort(X_train['Year'].unique())
n_splits = len(train_years) - 2
tscv = TimeSeriesSplit(n_splits=n_splits)

expanding_cv = []
for fold, (train_idx, val_idx) in enumerate(tscv.split(train_years[1:])):
    train_years_split = np.append(train_years[:1], train_years[train_idx + 1])
    val_years_split = train_years[val_idx + 1]
    
    train_indices = X_train[X_train['Year'].isin(train_years_split)].index
    val_indices = X_train[X_train['Year'].isin(val_years_split)].index
    expanding_cv.append((train_indices, val_indices))
    
    # Print the training and validation years for each fold
    print(f"Fold {fold+1}: Training Years: {train_years_split}, Validation Years: {val_years_split}")


# Initialize and perform grid search on ElasticNet
param_grid = {
    'alpha': [0],  
    'l1_ratio': [0]   # After extensive grid searching, no regularization turned out to perform the best
}
elastic_net = ElasticNet(random_state=42, max_iter=10000)
grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=expanding_cv, verbose=1)
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)


In [ ]:
# Predictions and inverse scaling back to the original Model-based Death Rate scale
y_train_pred_log = scaler_death_rate_target.inverse_transform(best_model.predict(X_train).reshape(-1, 1)).flatten()
y_test_pred_log = scaler_death_rate_target.inverse_transform(best_model.predict(X_test).reshape(-1, 1)).flatten()

# Apply exponential function to reverse log transformation
y_train_pred = np.exp(y_train_pred_log)
y_test_pred = np.exp(y_test_pred_log)

# Reverse transformations on actual target values for y_train and y_test
y_train_orig = np.exp(scaler_death_rate_target.inverse_transform(y_train).flatten())
y_test_orig = np.exp(scaler_death_rate_target.inverse_transform(y_test).flatten())

# Calculate metrics comparing predictions to the original Model-based Death Rate
train_metrics = {
    "MSE": mean_squared_error(y_train_orig, y_train_pred),
    "MAE": mean_absolute_error(y_train_orig, y_train_pred),
    "R-squared": r2_score(y_train_orig, y_train_pred),
    "MAPE": mean_absolute_percentage_error(y_train_orig, y_train_pred),
    "RMSE": np.sqrt(mean_squared_error(y_train_orig, y_train_pred))
}

test_metrics = {
    "MSE": mean_squared_error(y_test_orig, y_test_pred),
    "MAE": mean_absolute_error(y_test_orig, y_test_pred),
    "R-squared": r2_score(y_test_orig, y_test_pred),
    "MAPE": mean_absolute_percentage_error(y_test_orig, y_test_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test_orig, y_test_pred))
}

print("\nTraining Metrics:", train_metrics)
print("Testing Metrics:", test_metrics)

In [ ]:
# Calculate permutation importance on the training set
perm_importance_train = permutation_importance(best_model, X_train, y_train, n_repeats=10, random_state=42)

# Prepare data for plotting
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance (Mean)': perm_importance_train.importances_mean,
    'Importance (Std)': perm_importance_train.importances_std
}).sort_values(by='Importance (Mean)', ascending=False)

# Plot permutation feature importance
plt.figure(figsize=(10, 16))
plt.barh(feature_importances['Feature'], feature_importances['Importance (Mean)'], xerr=feature_importances['Importance (Std)'])
plt.xlabel("Permutation Importance (Mean Decrease in MSE)")
plt.ylabel("Features")
plt.title("Permutation Feature Importance (Linear Regression with 3 Lags and Log-Transformed Death Rates)")
plt.gca().invert_yaxis()  
plt.show()

In [ ]:
# For plotting the average actual and average predicted values per year for ElasticNet

# Use the ElasticNet test predictions and add them to the test DataFrame
test_df = test_df.copy()
test_df['ElasticNet_Predicted'] = y_test_pred  

# Group by Year and calculate the average actual and predicted death rates
yearly_avg_elastic_net = test_df.groupby('Year').agg({
    'Model-based Death Rate': 'mean',
    'ElasticNet_Predicted': 'mean'
})

# Plot the average actual and predicted values per year for ElasticNet
plt.figure(figsize=(10, 6))
plt.plot(yearly_avg_elastic_net.index, yearly_avg_elastic_net['Model-based Death Rate'], label='Actual', marker='o', color='b')
plt.plot(yearly_avg_elastic_net.index, yearly_avg_elastic_net['ElasticNet_Predicted'], label='Predicted', marker='o', color='r')
plt.xlabel('Year')
plt.ylabel('Average Death Rate')
plt.title('Average Actual vs Predicted Death Rates by Year (Linear Regression with 3 Lags and Log-Transformed Death Rates)')
plt.xticks(yearly_avg_elastic_net.index)  
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# XGBoost

# Required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess the dataset
file_path = 'NCHS_-_Drug_Poisoning_Mortality_by_County__United_States.csv'
df = pd.read_csv(file_path)

df = df.drop(columns=['FIPS State', 'FIPS', 'Standard Deviation', 'Lower Confidence Limit', 'Upper Confidence Limit'])
df = df.sort_values(by=['County', 'Year'])

# Create lag variables for 'Model-based Death Rate' and 'Population' (lag of 1 year)
df['Death_Rate_lag_1'] = df.groupby('County')['Model-based Death Rate'].shift(1)
df['Population_lag_1'] = df.groupby('County')['Population'].shift(1)

df = df.dropna(subset=['Death_Rate_lag_1', 'Population_lag_1'])

# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False, drop=None)

# Specify the categorical columns to encode
categorical_cols = ['State', 'Urban/Rural Category', 'Census Division']

# Apply one-hot encoding to the categorical columns
encoded_df = pd.DataFrame(onehot_encoder.fit_transform(df[categorical_cols]))
df = df.reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)
df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

# Select relevant features for exogenous variables (including lag variables)
exogenous_features = [col for col in df.columns if col not in ['Model-based Death Rate', 'County']]

# The target variable is still the 'Model-based Death Rate'
target_variable = 'Model-based Death Rate'



In [ ]:
# Train/Test Split
train_df = df[df['Year'].between(2004, 2017)]  
test_df = df[df['Year'].between(2018, 2021)]   

# Define features (X) and target (y) for train and test sets
X_train = train_df[exogenous_features]
y_train = train_df[target_variable]

X_test = test_df[exogenous_features]
y_test = test_df[target_variable]




In [ ]:
# Define custom scorers for GridSearchCV
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)  # RMSE
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)  # MAE
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)  # MAPE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=True)  # MSE
r2_scorer = make_scorer(r2_score)  # R²

# Initialize XGBoost regressor
xgb_model = xgb.XGBRegressor()

# Parameter grid for hyperparameter tuning (best performing hyperparameters found)
param_grid = {
    'n_estimators': [250],
    'learning_rate': [1.1],
    'max_depth': [17],
    'subsample': [1.0],
    'colsample_bytree': [1.0],
    'reg_alpha': [1.9],
    'reg_lambda': [1.9],
    'min_child_weight': [1],
    'gamma': [0.1]
}


X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Define expanding window cross-validation 
train_years = np.sort(X_train['Year'].unique())  
n_splits = len(train_years) - 2  
tscv = TimeSeriesSplit(n_splits=n_splits)

# Track the training and validation years for each fold
expanding_cv = []
for fold, (train_idx, val_idx) in enumerate(tscv.split(train_years[1:])):  # Start at the second year
    train_years_split = np.append(train_years[:1], train_years[train_idx + 1])
    val_years_split = train_years[val_idx + 1]
    
    train_indices = X_train[X_train['Year'].isin(train_years_split)].index
    val_indices = X_train[X_train['Year'].isin(val_years_split)].index
    
    expanding_cv.append((train_indices, val_indices))
    
    # Print the training and validation years for each fold
    print(f"Fold {fold+1}: Training Years: {train_years_split}, Validation Years: {val_years_split}")

# GridSearchCV setup and fitting
grid_search = GridSearchCV(
    estimator=xgb_model, 
    param_grid=param_grid, 
    scoring={
        'rmse': rmse_scorer, 
        'mae': mae_scorer, 
        'mape': mape_scorer, 
        'mse': mse_scorer, 
        'r2': r2_scorer
    }, 
    refit='mse',  # refit based on MSE
    cv=expanding_cv,  
    verbose=1
)

# Fit the model on the training data
grid_search.fit(X_train, y_train)

# Best parameters found
print("Best parameters found: ", grid_search.best_params_)

In [ ]:
# Predict on the training set
y_train_pred = grid_search.predict(X_train)

# Calculate and print metrics for the training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("Training Metrics:")
print(f"Train MSE: {train_mse}")
print(f"Train RMSE: {train_rmse}")
print(f"Train MAE: {train_mae}")
print(f"Train MAPE: {train_mape * 100}%")
print(f"Train R²: {train_r2}")

In [ ]:
# Predict on the test set
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)

# Evaluate performance on the test set
mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Test MAE: {mae_test}")
print(f"Test RMSE: {rmse_test}")
print(f"Test MSE: {mse_test}")
print(f"Test MAPE: {mape_test * 100}%")
print(f"Test R²: {r2_test}")

In [ ]:
# LightGBM

# Required Libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import lightgbm as lgb
import matplotlib.pyplot as plt


In [ ]:
# Load the dataset and preprocess
file_path = 'NCHS_-_Drug_Poisoning_Mortality_by_County__United_States.csv'
df = pd.read_csv(file_path)

df = df.drop(columns=['FIPS State', 'FIPS', 'Standard Deviation', 'Lower Confidence Limit', 'Upper Confidence Limit'])
df = df.sort_values(by=['Year', 'County'])

# Manually rename the 'Urban/Rural Category' and 'Census Division' features to remove white space
df = df.rename(columns={
    'Urban/Rural Category': 'Urban_Rural_Category',
    'Model-based Death Rate': 'Model_based_Death_Rate',
    'Census Division': 'Census_Division'
})


# Create lag variables for 'Population' and 'Model-based Death Rate'
df['Population_lag_1'] = df.groupby('County')['Population'].shift(1)
df['DeathRate_lag_1'] = df.groupby('County')['Model_based_Death_Rate'].shift(1)

df['Population_lag_2'] = df.groupby('County')['Population'].shift(2)
df['DeathRate_lag_2'] = df.groupby('County')['Model_based_Death_Rate'].shift(2)

df['Population_lag_3'] = df.groupby('County')['Population'].shift(3)
df['DeathRate_lag_3'] = df.groupby('County')['Model_based_Death_Rate'].shift(3)


# Drop any rows with missing values created by lag
df = df.dropna()

# Specify the categorical columns 
categorical_cols = ['State', 'Urban_Rural_Category', 'Census_Division']

target_variable = 'Model_based_Death_Rate'

# Select relevant features for exogenous variables, including lag features
exogenous_features = [col for col in df.columns if col not in ['Model_based_Death_Rate', 'County']]

df[categorical_cols] = df[categorical_cols].astype('category')


In [ ]:
# Train/Test Split
train_df = df[df['Year'].between(2006, 2017)]  
test_df = df[df['Year'].between(2018, 2021)]   

# Define features (X) and target (y) for train and test sets
X_train = train_df[exogenous_features]
y_train = train_df[target_variable]

X_test = test_df[exogenous_features]
y_test = test_df[target_variable]


In [ ]:
# Initialize LightGBM regressor
lgb_model = lgb.LGBMRegressor(random_state=42)

# Parameter grid for tuning (best-performing hyperparameters found) 
param_grid = {
    'n_estimators': [500],
    'learning_rate': [1],
    'max_depth': [17],
    'subsample': [1.0],
    'colsample_bytree': [1.0],
    'reg_alpha': [2.2],
    'reg_lambda': [2.7],
    'min_child_weight': [70],
}


# Convert categorical columns to 'category' dtype
for col in categorical_cols:
    X_train[col] = X_train[col].astype('category')

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Define expanding window cross-validation with at least two years in the first fold
train_years = np.sort(X_train['Year'].unique())  
n_splits = len(train_years) - 2  
tscv = TimeSeriesSplit(n_splits=n_splits)

# Track the training and validation years for each fold
expanding_cv = []
for fold, (train_idx, val_idx) in enumerate(tscv.split(train_years[1:])):  # Start at the second year
    train_years_split = np.append(train_years[:1], train_years[train_idx + 1])
    val_years_split = train_years[val_idx + 1]
    
    train_indices = X_train[X_train['Year'].isin(train_years_split)].index
    val_indices = X_train[X_train['Year'].isin(val_years_split)].index
    
    expanding_cv.append((train_indices, val_indices))
    
    # Print the training and validation years for each fold
    print(f"Fold {fold+1}: Training Years: {train_years_split}, Validation Years: {val_years_split}")

# Define custom scorers for GridSearchCV
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)  # RMSE
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)  # MAE
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)  # MAPE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=True)  # MSE
r2_scorer = make_scorer(r2_score)  # R²

# GridSearchCV for hyperparameter tuning 
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, 
                           scoring={'rmse': rmse_scorer, 'mae': mae_scorer, 'mape': mape_scorer, 'mse': mse_scorer, 'r2': r2_scorer}, 
                           refit='mse',  # Refit based on MSE
                           cv=expanding_cv,  
                           verbose=0)

# Fit the model on the training data, passing categorical feature names
grid_search.fit(X_train, y_train, categorical_feature=categorical_cols)

print("Best parameters found by grid search:", grid_search.best_params_)


In [ ]:
# Predict on the training set
y_train_pred = grid_search.predict(X_train)

# Calculate and print metrics for the training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("Training Metrics:")
print(f"Train MSE: {train_mse}")
print(f"Train RMSE: {train_rmse}")
print(f"Train MAE: {train_mae}")
print(f"Train MAPE: {train_mape * 100}%")
print(f"Train R²: {train_r2}")

In [ ]:
# Evaluate on the test set
y_pred = grid_search.predict(X_test)

# Calculate and print metrics for the test set
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_pred)
test_mape = mean_absolute_percentage_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test MAPE: {test_mape * 100}%")
print(f"Test R²: {test_r2}")